# Use Case Evaluation

## Connecting to the CAOM postgres database

In [1]:
import psycopg2
import pandas as pd

In [2]:
import pyvo as vo

In [5]:
service = vo.dal.TAPService("http://localhost:8080/tap")

In [6]:
resultset = service.search("SELECT * FROM TAP_SCHEMA.tables WHERE schema_name='public'")

In [14]:
range(1,16)

range(1, 16)

In [16]:
for x in range(1,17):
    print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


## Executing SQL Queries via the connection

Returning all observations from the database

In [9]:
resultset = service.search("SELECT * FROM Observation")

In [10]:
resultset.to_table()

algorithm_id,collection,environment_id,id,instrument_id,intent,metareadgroups,metarelease,proposal_id,requirements_id,sequencenumber,target_id,targetposition_id,telescope_id,type,uri,uribucket
int64,object,int64,object,int64,object,object,object,int64,int64,int32,int64,int64,int64,object,object,object
1,EMERLIN,--,ae24354f-5e66-4d47-9ed9-410876332ff2,1,SCIENCE,,,--,--,--,--,--,1,,CY14205_C_001_20220901_Mk2,6e8
2,EMERLIN,--,eec0b688-7383-4e19-ba68-b29409cb40dc,2,SCIENCE,,,--,--,--,--,--,2,,CY14205_C_001_20220901_Kn,abc
3,EMERLIN,--,7f038b21-eb45-47cd-a73b-dcf9d333b7e0,3,SCIENCE,,,--,--,--,--,--,3,,CY14205_C_001_20220901_De,22a
4,EMERLIN,--,62c37eb8-39a9-4e4c-a6fb-30d2f86be38e,4,SCIENCE,,,--,--,--,--,--,4,,CY14205_C_001_20220901_Pi,d5d
5,EMERLIN,--,56bfc1d0-4634-4149-b3dc-b8ee1d15f28f,5,SCIENCE,,,--,--,--,--,--,5,,CY14205_C_001_20220901_Da,99d
6,EMERLIN,--,e2ef63eb-89b0-4b25-bfaa-16ca3dcd4c3f,6,SCIENCE,,,--,--,--,--,--,6,,CY14205_C_001_20220901_Cm,43c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,EMERLIN,--,a053ed1b-5ddb-466c-accc-76d28a4821f1,--,SCIENCE,,,--,--,--,4,4,--,,CY14205_C_001_20220901_002913+131604,caa
11,EMERLIN,--,a685d4d0-cf87-465b-b179-e1277063f303,--,SCIENCE,,,--,--,--,5,5,--,,CY14205_C_001_20220901_1331+3030,fd9


In [11]:
len(resultset.to_table())

16

In [157]:
resultset = service.search("SELECT * FROM derivedobservation")

In [158]:
resultset.to_table()

id,members
object,object
d65aeffc-6dc2-40d3-a590-caba61d84613,CY16204_C_001_20231117_Cm;CY16204_C_001_20231117_Mk2;CY16204_C_001_20231117_1331+3030;CY16204_C_001_20231117_1407+2827;CY16204_C_001_20231117_Kn;CY16204_C_001_20231117_Pi;CY16204_C_001_20231117_De;CY16204_C_001_20231117_Da;CY16204_C_001_20231117_1415+1320
d8f3c78b-edc3-417f-b1ac-a2bf9411e4ef,CY14205_C_001_20220901_0035+1438;CY14205_C_001_20220901_005334+124136;CY14205_C_001_20220901_Kn;CY14205_C_001_20220901_015950+002340;CY14205_C_001_20220901_1331+3030;CY14205_C_001_20220901_Cm;CY14205_C_001_20220901_0319+4130;CY14205_C_001_20220901_Mk2;CY14205_C_001_20220901_Da;CY14205_C_001_20220901_0157+0011;CY14205_C_001_20220901_De;CY14205_C_001_20220901_002913+131604;CY14205_C_001_20220901_1407+2827;CY14205_C_001_20220901_0055+1408;CY14205_C_001_20220901_Pi


In [159]:
derived_observation_id = resultset.to_table()['id'][0]

## Use Cases for e-Merlin Metadata

### Use Case 1 
As an astronomer, I want to be able to browse the e-Merlin archive so that I can find data products that are suitable for my science goals.

Observational metadata including: target (name, id, etc.), WCS information (RA/DEC), date/time of
observation, observing frequency, spectral line data, baseline, resolution, beam, field of view, PI or researchers, Telescopes
present, number of visits, Polarisation

In [358]:
resultset = service.search("""
SELECT t.*, i.* FROM Telescope t 
JOIN Observation o ON o.telescope_id = t.id 
JOIN Instrument i ON i.id = o.instrument_id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )

;""".format(derived_observation_id))
# only returns location of sites, calculation for baseline is paused until issues with the values are resolved

In [359]:
resultset.to_table() # add determination of baseline from geolocation after checking it

geolocationx,geolocationy,geolocationz,id,keywords,name,trackingmode,id2,keywords2,name2
float64,float64,float64,int64,object,object,object,int64,object,object
0.018163180067862095,-0.026508953212361644,-0.024171788669066677,1,,e-MERLIN,,1,,Mk2
-0.00844229087230352,0.04069186441719252,-0.03176831208560866,2,,e-MERLIN,,2,,Kn
-0.014581808067785246,0.025596742665565733,-0.02308715833180284,3,,e-MERLIN,,3,,De
0.012334199619778453,-0.03182568604765,-0.02562333464060841,4,,e-MERLIN,,4,,Pi
0.010207504870832346,0.03382407987219867,-0.02665123418525062,5,,e-MERLIN,,5,,Da
0.0005105876155187977,1.5140529300510745e-05,0.0003993721621628234,6,,e-MERLIN,,6,,Cm


In [135]:
resultset = service.search("""
SELECT t.name, p.* FROM Target t 
JOIN Observation o ON o.target_id = t.id 
JOIN point p ON t.id = p.id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )

;""".format(derived_observation_id))

In [136]:
resultset.to_table()

name,cval1,cval2,id,polygon_id
object,float64,float64,int64,int64
1331+3030,202.78453041666666,30.509155277777786,1,--
1407+2827,211.751643375,28.454080527777776,2,--
1415+1320,213.9950708333333,13.339920000000003,3,--


In [150]:
resultset = service.search("""
SELECT pl.*, t.bounds_upper, t.bounds_lower, e.bounds_upper, e.bounds_lower, p.states, pvn.project FROM Plane pl 
JOIN Observation o ON o.id = pl.observation_id 
JOIN time t ON pl.time_id = t.id
JOIN energy e ON pl.energy_id = e.id
JOIN polarization p ON pl.polarization_id = p.id
JOIN provenance pvn ON pl.provenance_id = pvn.id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )
;""".format(derived_observation_id))

In [151]:
resultset.to_table()

calibrationlevel,custom_id,dataproducttype,datareadgroups,datarelease,energy_id,id,metareadgroups,metarelease,metrics_id,observable_id,observation_id,polarization_id,position_id,provenance_id,quality_id,time_id,uri,uv_id,bounds_upper,bounds_lower,bounds_upper2,bounds_lower2,states,project
object,int64,object,object,object,int64,object,object,object,int64,int64,object,int64,int64,int64,int64,int64,object,int64,float64,float64,float64,float64,object,object
,--,,,2023-11-17T10:45:05Z,1,a7483f68-c88b-4495-9315-d8c1a4599ea8,,,--,--,d65aeffc-6dc2-40d3-a590-caba61d84613,1,--,3,--,1,1415+1320,--,5206939798.0,5206939206.5,0.062236341706456304,0.056277915900131405,"{RR,LL}",CY16204
,--,visibility,,2023-11-17T10:45:05Z,2,f40ed4dd-9490-4561-b733-a934cc9849f4,,,--,--,d65aeffc-6dc2-40d3-a590-caba61d84613,2,--,4,--,2,CY16204_C_001_20231117_avg.ms,--,5206939798.0,5206934706.5,0.06224280244991176,0.056272634068512434,"{RR,LL}",CY16204


In [165]:
resultset = service.search("""
SELECT pl.* FROM Plane pl
WHERE pl.dataproducttype = 'spectrum'
""")

In [166]:
resultset.to_table()

calibrationlevel,custom_id,dataproducttype,datareadgroups,datarelease,energy_id,id,metareadgroups,metarelease,metrics_id,observable_id,observation_id,polarization_id,position_id,provenance_id,quality_id,time_id,uri,uv_id
object,int64,object,object,object,int64,object,object,object,int64,int64,object,int64,int64,int64,int64,int64,object,int64


In [174]:
resultset = service.search("""
SELECT pl.resolution, pl.resolutionbounds_lower, pl.resolutionbounds_upper FROM position pl
""")

In [175]:
resultset.to_table()

resolution,resolutionbounds_lower,resolutionbounds_upper
float64,float64,float64


In [ ]:
 ### These not currently included: beam, field of view (revisit spectrum and resolution and beam)

### Use Case 2 
As an astronomer, I want browse the quality assessment information associated with the retrieved data products.

Quality from the raw data table (perhaps not populated), quality in the processing (this page as a starting
point for failed steps) Requested or predicted sensitivity, Telescopes scheduled, Comments on observing, other information
used as indicators of quality e.g. UV plane

### Use Case 3
As an astronomer, track the workflow history of a data product so that I can understand how it was generated by the Observatory.

Pipeline version, input parameters, raw data identifier


In [186]:
resultset = service.search("""
SELECT p.* FROM provenance p
JOIN Plane pl ON pl.provenance_id = p.id 
JOIN Observation o ON o.id = pl.observation_id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )
""".format(derived_observation_id))

In [187]:
resultset.to_table()

id,inputs,keywords,lastexecuted,name,producer,project,reference,runid,version
int64,object,object,object,object,object,object,object,object,object
2,,Role pointing_calibrator,,eMERLIN_CASA_pipeline,,,,CY16204_C_001_20231117,v1.1.19
3,,,,eMERLIN_CASA_pipeline,,CY16204,,CY16204_C_001_20231117,v1.1.19
4,,,,eMERLIN_CASA_pipeline,,CY16204,,CY16204_C_001_20231117,v1.1.19
1,,Role flux_calibrator,,eMERLIN_CASA_pipeline,,,,CY16204_C_001_20231117,v1.1.19


### Use Case 4

As an astronomer, I want to be able to find data that is closely linked to my dataset. For example, data
that shares a project code, principle investigator, or was derived from the same set of raw data.

Project code, PI, raw data identifier

In [189]:
resultset = service.search("""
SELECT p.project, p.producer, o.uri FROM provenance p
JOIN Plane pl ON pl.provenance_id = p.id 
JOIN Observation o ON o.id = pl.observation_id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )
""".format(derived_observation_id))

In [190]:
resultset.to_table()

project,producer,uri
object,object,object
,,CY16204_C_001_20231117
CY16204,,CY16204_C_001_20231117
CY16204,,CY16204_C_001_20231117
,,CY16204_C_001_20231117


### Use Case 5
As an astronomer, I want to be able to use geometric spatial searches for my target data (cone search etc.)

PostgresSQL, WCS info, field of view

In [ ]:
resultset = vo.dal.conesearch("http://localhost:8080/tap", )

In [17]:
# SELECT o.uri, DISTANCE(202.65, 30.51, p.cval1, p.cval2) AS ang_sep

resultset = service.search("""
SELECT o.uri, p.cval1, p.cval2 
FROM point p
JOIN targetposition tp ON tp.id = p.id
JOIN observation o ON tp.id = o.targetposition_id
WHERE p.cval1 > 202. AND p.cval1 < 203. AND p.cval2 > 30. AND p.cval2 < 31.
""")

In [18]:
resultset.to_table()

uri,cval1,cval2
object,float64,float64
CY14205_C_001_20220901_1331+3030,202.78453041666666,30.509155277777786


In [230]:
# SELECT o.uri, DISTANCE(202.65, 30.51, p.cval1, p.cval2) AS ang_sep

# possible methods for cone search which are stated in the pyvo documentation but do not work 

# resultset = service.search("""
# SELECT o.uri, p.cval1, p.cval2 
# FROM point p
# JOIN targetposition tp ON tp.id = p.id
# JOIN observation o ON tp.id = o.targetposition_id
# WHERE DISTANCE(POINT('ICRS', 201.,30.) POINT('ICRS', 201.,30.))<0.01
# """)

# resultset = service.search("""
# SELECT o.uri, p.cval1, p.cval2 
# FROM point p
# JOIN targetposition tp ON tp.id = p.id
# JOIN observation o ON tp.id = o.targetposition_id
# WHERE CONTAINS(POINT('ICRS', 201.,30.) CIRCLE('ICRS', 201.,30.,0.05))=1
# """)

### Use Case 6 

As an astronomer I want to be able to search for ancillary sources to my target data and discover their
roles.

Ancillary target names and roles

In [267]:
resultset = service.search("""
SELECT p.observation_id, p.uri, pr.keywords FROM Plane p
JOIN Provenance pr ON p.provenance_id = pr.id
""")

In [268]:
resultset.to_table()

observation_id,uri,keywords
object,object,object
d65aeffc-6dc2-40d3-a590-caba61d84613,1331+3030,Role flux_calibrator
d65aeffc-6dc2-40d3-a590-caba61d84613,1407+2827,Role pointing_calibrator
d65aeffc-6dc2-40d3-a590-caba61d84613,1415+1320,
d65aeffc-6dc2-40d3-a590-caba61d84613,CY16204_C_001_20231117_avg.ms,
d8f3c78b-edc3-417f-b1ac-a2bf9411e4ef,005334+124136,
d8f3c78b-edc3-417f-b1ac-a2bf9411e4ef,0319+4130,Role pointing_calibrator
d8f3c78b-edc3-417f-b1ac-a2bf9411e4ef,0157+0011,Role phase_calibrator_2
d8f3c78b-edc3-417f-b1ac-a2bf9411e4ef,002913+131604,
d8f3c78b-edc3-417f-b1ac-a2bf9411e4ef,1331+3030,Role flux_calibrator


### Use Case 7 

As an operator I want to be able to re-compute any data product based upon the metadata.

Pipeline used, versions, parameters, raw data access, raw data location, data product access, data product
location (emerlin measurement set directory structure includes .ini file and casa logs), release date, runtime environment

In [292]:
# raw data access, raw data location, release date?

resultset = service.search("""

SELECT o.uri, pr.name, pr.version, p.datarelease, a.uri, a.contenttype, a.contentrelease FROM Observation o 
JOIN Plane p ON o.id=p.observation_id
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
""")

In [293]:
resultset.to_table()

uri,name,version,datarelease,uri2,contenttype,contentrelease
object,object,object,object,object,object,object
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png,image/png,
...,...,...,...,...,...,...
CY14205_C_001_20220901,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png,image/png,
CY14205_C_001_20220901,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png,image/png,


### Use Case 8 

As an operator I want to be able to determine if any given data product should be stored or re-computed
on demand. 

Available resources (storage and compute), memory consumption of pipeline, process timings, data (physical)
size, number of backups necessary, number of times data is accessed, raw data location/identifiers

In [298]:
resultset = service.search("""
SELECT pr.lastexecuted, a.contentlength, a.uri FROM Plane p
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
""")

# not currently included: available resources, mem consumption, processing timing, backups, accesses

In [299]:
resultset.to_table()

lastexecuted,contentlength,uri
object,int32,object
,573923,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png
,308239,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png
,237693,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png
,242288,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png
,374433,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png
,55698,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png
...,...,...
,243624,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png
,351805,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png


### Use Case 9 

As an operator I want to be able to detect anomalies within the data processing or data itself.

Quality metrics, parameters, processing time, memory consumption, identifiers, raw data links,

In [304]:
resultset = service.search("""
SELECT pr.lastexecuted, a.contentlength, a.uri FROM Plane p
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
""")

# missing quality, parameters

In [305]:
resultset.to_table()

lastexecuted,contentlength,uri
object,int32,object
,573923,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png
,308239,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png
,237693,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png
,242288,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png
,374433,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png
,55698,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png
...,...,...
,243624,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png
,351805,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png


### Use Case 10

As an operator I want to be able to determine if the needs of a proposal can be met by existing data products

Observational metadata (date/time, location, field of view, polarisation, frequency, quality, etc.)

In [310]:
resultset = service.search("""
SELECT pl.*, t.bounds_upper, t.bounds_lower, e.bounds_upper, e.bounds_lower, p.states, pvn.project FROM Plane pl 
JOIN Observation o ON o.id = pl.observation_id 
JOIN time t ON pl.time_id = t.id
JOIN energy e ON pl.energy_id = e.id
JOIN polarization p ON pl.polarization_id = p.id
JOIN provenance pvn ON pl.provenance_id = pvn.id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )
;""".format(derived_observation_id))

In [311]:
resultset.to_table()

calibrationlevel,custom_id,dataproducttype,datareadgroups,datarelease,energy_id,id,metareadgroups,metarelease,metrics_id,observable_id,observation_id,polarization_id,position_id,provenance_id,quality_id,time_id,uri,uv_id,bounds_upper,bounds_lower,bounds_upper2,bounds_lower2,states,project
object,int64,object,object,object,int64,object,object,object,int64,int64,object,int64,int64,int64,int64,int64,object,int64,float64,float64,float64,float64,object,object
,--,,,2023-11-17T10:45:05Z,1,a7483f68-c88b-4495-9315-d8c1a4599ea8,,,--,--,d65aeffc-6dc2-40d3-a590-caba61d84613,1,--,3,--,1,1415+1320,--,5206939798.0,5206939206.5,0.062236341706456304,0.056277915900131405,"{RR,LL}",CY16204
,--,visibility,,2023-11-17T10:45:05Z,2,f40ed4dd-9490-4561-b733-a934cc9849f4,,,--,--,d65aeffc-6dc2-40d3-a590-caba61d84613,2,--,4,--,2,CY16204_C_001_20231117_avg.ms,--,5206939798.0,5206934706.5,0.06224280244991176,0.056272634068512434,"{RR,LL}",CY16204


### Use Case 11 

As an operator, I want to be able to selectively determine when and to whom data should available

Permissions status, time/date, project code, user information, agreements info (if they can be different),
release date, meta release date

In [333]:
resultset = service.search("""
SELECT pr.lastexecuted, a.uri, pr.project, p.datarelease, p.metarelease, o.metarelease FROM Plane p
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
JOIN Observation o ON o.id = p.observation_id
"""

In [334]:
resultset.to_table()

lastexecuted,uri,project,datarelease,metarelease,metarelease2
object,object,object,object,object,object
,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png,,,,
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png,,,,
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png,,,,
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png,,,,
,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png,,,,
,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png,,,,
...,...,...,...,...,...
,uri:CY16204_C_001_20231117_avg.ms,CY16204,2023-11-17T10:45:05Z,,
,uri:CY16204_C_001_20231117_avg_uvcov_1407+2827.png,,,,


### Use Case 12

As an operator, I want to be able to infer data quality from available metrics

Resolution, weather, time/date, elevation, pipeline version/name/provenance, beam shape, uv coverage,
max/min baseline length, est or actual sensitivity

In [340]:
resultset = service.search("""
SELECT o.id, t.bounds_lower, t.bounds_upper, pr.name, pr.version FROM Observation o 
JOIN Plane p ON o.id = p.observation_id
JOIN time t ON t.id = p.time_id
JOIN Provenance pr ON pr.id = provenance_id
""")

# time/date, pipeline name/version, baseline
# add baseline query up top when completed

In [338]:
resultset.to_table()

id,bounds_lower,bounds_upper,name,version
object,float64,float64,object,object
d65aeffc-6dc2-40d3-a590-caba61d84613,5206939206.5,5206939798.0,eMERLIN_CASA_pipeline,v1.1.19
d65aeffc-6dc2-40d3-a590-caba61d84613,5206934706.5,5206939798.0,eMERLIN_CASA_pipeline,v1.1.19


In [362]:
resultset = service.search("""
SELECT o.id, o.uri, t.geolocationx, t.geolocationy, t.geolocationz, i.name FROM Observation o
JOIN Telescope t ON t.id = o.telescope_id
JOIN Instrument i ON i.id = o.instrument_id
WHERE EXISTS (
    SELECT 1
    FROM Derivedobservation d 
    WHERE d.id = '{}' AND d.members LIKE '%' || o.uri || '%'
    )
""".format(derived_observation_id))

# only returns location of sites, calculation for baseline is paused until issues with the values are resolved

In [363]:
resultset.to_table()

id,uri,geolocationx,geolocationy,geolocationz,name
object,object,float64,float64,float64,object
45e7d7ae-e771-40ca-bd6a-bd11a6d22331,CY16204_C_001_20231117_Mk2,0.018163180067862095,-0.026508953212361644,-0.024171788669066677,Mk2
6e08aa0d-e869-4833-9765-f6aa21c308ec,CY16204_C_001_20231117_Kn,-0.00844229087230352,0.04069186441719252,-0.03176831208560866,Kn
47a4405f-3080-4aca-bd75-708528542ef3,CY16204_C_001_20231117_De,-0.014581808067785246,0.025596742665565733,-0.02308715833180284,De
ab867ede-2aeb-4409-a43a-421cff875c25,CY16204_C_001_20231117_Pi,0.012334199619778453,-0.03182568604765,-0.02562333464060841,Pi
3097a3c9-0328-4773-91fb-ba620e314630,CY16204_C_001_20231117_Da,0.010207504870832346,0.03382407987219867,-0.02665123418525062,Da
e38d1372-15a3-4731-a16f-c499dcfd201f,CY16204_C_001_20231117_Cm,0.0005105876155187977,1.5140529300510745e-05,0.0003993721621628234,Cm


### Use Case 13 

As a computer, I want to be able to process observational data without human interaction.

Raw data location/identifiers, pipeline info (location, version, available deployments), included anten-
nas, correlator setup, calibrator choices, calibrator catalog, storage/compute resource availability, storage location, proposal database access, PI, target name, WCS, casa access

In [352]:
resultset = service.search("""
SELECT o.uri, pr.name, pr.version, p.datarelease, a.uri, a.contenttype, a.contentrelease, pr.project
FROM Observation o 
JOIN Plane p ON o.id=p.observation_id
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
""")

# artifact uri, pipeline name/version, antenna names (just make the query), calibrator roles + names, project code, 
# target name

In [353]:
resultset.to_table()

uri,name,version,datarelease,uri2,contenttype,contentrelease,project
object,object,object,object,object,object,object,object
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png,image/png,,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png,image/png,,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png,image/png,,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png,image/png,,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png,image/png,,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png,image/png,,
...,...,...,...,...,...,...,...
CY14205_C_001_20220901,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png,image/png,,
CY14205_C_001_20220901,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png,image/png,,


### Use Case 14

As a computer, I want to be able to reprocess data products in the case of poor data quality or anoma-
lous results.

See use case 13, Raw data location/identifiers, pipeline info (location, version, available deployments), quality/success of data processing, information to create a baseline of expected results (possibly target name and identifiers/locations to associated data products themselves for comparison

In [318]:
resultset = service.search("""
SELECT o.uri, pr.name, pr.version, p.datarelease, a.uri, a.contenttype, a.contentrelease FROM Observation o 
JOIN Plane p ON o.id=p.observation_id
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
""")

# not included - quality. Can add when it is in the db

In [319]:
resultset.to_table()

uri,name,version,datarelease,uri2,contenttype,contentrelease
object,object,object,object,object,object,object
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png,image/png,
CY16204_C_001_20231117,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png,image/png,
...,...,...,...,...,...,...
CY14205_C_001_20220901,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png,image/png,
CY14205_C_001_20220901,eMERLIN_CASA_pipeline,v1.1.19,,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png,image/png,


### Use Case 15

As a computer, I want to prevent proprietary science data products and certain metadata from being ac-
cessed by unauthorised users, and allow public data to be authorised and publicly searchable by any users.

Release date, ObsID/observation identifier/projectID, PI name, authentication credentials, public/proprietary
flag

In [346]:
resultset = service.search("""
SELECT pr.lastexecuted, a.uri, pr.project, p.datarelease, p.metarelease, o.metarelease, o.intent FROM Plane p
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
JOIN Observation o ON o.id = p.observation_id
""")

In [347]:
resultset.to_table()

lastexecuted,uri,project,datarelease,metarelease,metarelease2,intent
object,object,object,object,object,object,object
,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png,,,,,SCIENCE
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png,,,,,SCIENCE
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png,,,,,SCIENCE
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png,,,,,SCIENCE
,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png,,,,,SCIENCE
,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png,,,,,SCIENCE
...,...,...,...,...,...,...
,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png,,,,,SCIENCE
,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png,,,,,SCIENCE


### Use Case 16

As a computer, I want to notice new data products and ingest them into a database.

ObsID/run/date file path component query on database, code to check for new products, code to generate
metadata to populate db tables

In [348]:
resultset = service.search("""
SELECT pr.lastexecuted, a.uri, pr.project, p.datarelease FROM Plane p
JOIN Provenance pr ON pr.id = p.provenance_id
JOIN Artifact a ON a.plane_id = p.id
JOIN Observation o ON o.id = p.observation_id
""")

In [349]:
resultset.to_table()

lastexecuted,uri,project,datarelease
object,object,object,object
,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_data1.png,,
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected0.png,,
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected1.png,,
,uri:CY16204_C_001_20231117_avg_4plot_1331+3030_corrected3.png,,
,uri:CY16204_C_001_20231117_avg_4plot_1407+2827_corrected0.png,,
,uri:CY16204_C_001_20231117_avg_uvpltmodel_1407+2827.png,,
...,...,...,...
,uri:CY14205_C_001_20220901_avg_4plot_1407+2827_corrected3.png,,
,uri:CY14205_C_001_20220901_avg_4plot_0319+4130_corrected0.png,,
